In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))
from os.path import expanduser

%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
load_dotenv()

from google.cloud import storage
from project_config import GCP_PROJECT_NAME, DATASET_JSON_PATH

gcp_client = storage.Client(project=GCP_PROJECT_NAME)

/home/ando/.conda/envs/rv_timm0412/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


### Configuration

In [2]:
from experiment_configs.configs import *
config = satmae_large_config

### Create Rastervision datasets

In [3]:
from torch.utils.data import ConcatDataset
import json
from utils.rastervision_pipeline import observation_to_scene, scene_to_training_ds, scene_to_validation_ds, warn_if_nan_in_raw_raster
from utils.data_management import observation_factory, characterize_dataset
import random

#set the seed
random.seed(13)

# get the current working directory
root_dir = os.getcwd()

# define the relative path to the dataset JSON file
json_rel_path = '../' + DATASET_JSON_PATH

# combine the root directory with the relative path
json_abs_path = os.path.join(root_dir, json_rel_path)

dataset_json = json.load(open(json_abs_path, 'r'))
all_observations = observation_factory(dataset_json)

#find the highest cluster id
max_cluster_id = max([observation['cluster_id'] for observation in dataset_json])


# Randomly split the data into training and validation
val_split = random.randint(0, max_cluster_id+1)

training_scenes = []
validation_scenes = []

for observation in all_observations:
    if observation.cluster_id == val_split: 
        validation_scenes.append(observation_to_scene(config, observation))
    else:
        training_scenes.append(observation_to_scene(config, observation))


training_datasets = [
    scene_to_training_ds(config, scene) for scene in training_scenes #random window sampling happens here
]
validation_datasets = [
    scene_to_validation_ds(config, scene) for scene in validation_scenes
]

train_dataset_merged = ConcatDataset(training_datasets)
val_dataset_merged = ConcatDataset(validation_datasets)

print('Validation split cluster_id:', val_split)
print ('Training dataset size: {:4d} images | Number of observations: {:}'.format(len(train_dataset_merged), len(training_scenes)))
print ('Testing dataset size: {:4d}  images | Number of observations: {:}'.format(len(val_dataset_merged), len(validation_scenes)))

mine_percentage_aoi = characterize_dataset(training_scenes, validation_scenes)

2023-12-14 19:31:14:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/s2/Kathajodi_Cuttack_85-85_20-44_2022-05-01_s2.tif.
2023-12-14 19:31:14:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/annotations/Kathajodi_Cuttack_85-85_20-44_2022-05-01_annotations.geojson.


2023-12-14 19:31:14:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/rivers/Kathajodi_Cuttack_85-85_20-44_rivers_1000m.geojson.
2023-12-14 19:31:14:rastervision.core.data.vector_source.geojson_vector_source: INFO - Ignoring CRS ({'type': 'name', 'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}}) specified in https://storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/rivers/Kathajodi_Cuttack_85-85_20-44_rivers_1000m.geojson and assuming EPSG:4326 instead.
2023-12-14 19:31:14:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/sand_mining_median/labels/Kathajodi_Cuttack_85-85_20-44_median/s2/Kathajodi_Cuttack_85-85_20-44_2022-02-01_s2.tif.
2023-12-14 19:31:14:rastervision.pipeline.file_system.utils: INFO - Using cached file /data/tmp/cache/http/storage.googleapis.com/

Validation split cluster_id: 4
Training dataset size: 3088 images | Number of observations: 72
Testing dataset size:  378  images | Number of observations: 8
Total dataset has 2.93%  mining area.
Training dataset has 3.05%  mining area.
Validation dataset has 2.15%  mining area.
Within AOIs, total dataset has 6.72%  mining area.
Outside AOIs, total dataset has 0.06%  mining area.

The median percentage of mine in an observation is 3.26%
The median number of mine pixels in an observation is 38059

The median number pixels in an observation is 1301008


## Update the loss weights to account for the imbalanced dataset

In [4]:
config.mine_class_loss_weight = (100 - mine_percentage_aoi) / mine_percentage_aoi
config.mine_class_loss_weight

13.880675775871904

## Visualize the datasets

In [5]:
from utils.visualizing import visualize_dataset

# print ('Training Dataset')
# for ds in training_datasets:
#     visualize_dataset(ds)

# print("\n\n\n Val Dataset")

# for ds in validation_datasets:
#     visualize_dataset(ds)

# Train

In [6]:
from models.model_factory import model_factory
from ml.optimizer_factory import optimizer_factory
from ml.learner import BinarySegmentationLearner

_, _, n_channels = training_datasets[0].scene.raster_source.shape
model = model_factory(
    config,
    n_channels=n_channels,
)

optimizer = optimizer_factory(config, model)

learner = BinarySegmentationLearner(
    config=config,
    model=model,
    optimizer=optimizer,
    train_ds=train_dataset_merged,  # for development and debugging, use training_datasets[0] or similar to speed up
    valid_ds=val_dataset_merged,  # for development and debugging, use training_datasets[1] or similar to speed up
    output_dir=expanduser("~/sandmining-watch/out/OUTPUT_DIR"),
)

learner.log_data_stats()

SatMae: Loading encoder weights from /data/sand_mining/checkpoints/satmae_orig/pretrain-vit-large-e199.pth
Position interpolate from 12x12 to 20x20
Warning! Missing keys:
['channel_cls_embed', 'head.weight', 'head.bias']
SatMaePretrained: Freezing encoder weights
Number of parameters loaded: 298


2023-12-14 19:31:32:rastervision: INFO - train_ds: 3088 items
2023-12-14 19:31:32:rastervision: INFO - valid_ds: 378 items


#### Check GPU Activity

You can continuously monitor your GPU activity by using the command in the terminal


`watch -d -n 0.5 nvidia-smi`

In [7]:
# Run this cell if you want to log the run to W&B. You might need to authenticate to W&B.
learner.initialize_wandb_run()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andoshah (sandmining-watch). Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [8]:
learner.train(epochs=40)

2023-12-14 19:32:09:rastervision: INFO - epoch: 0


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

2023-12-14 19:38:04:rastervision: INFO - metrics:
{'avg_f1': 0.8204631805419922,
 'avg_precision': 0.9686272144317627,
 'avg_recall': 0.7116127610206604,
 'epoch': 0,
 'other_f1': 0.825176477432251,
 'other_precision': 0.9978607892990112,
 'other_recall': 0.7034425735473633,
 'sandmine_average_precision': 0.5013793910147574,
 'sandmine_best_f1_score': 0.4771922480407363,
 'sandmine_best_threshold': 0.9330304,
 'sandmine_f1': 0.17699861526489258,
 'sandmine_precision': 0.09753743559122086,
 'sandmine_recall': 0.9550645351409912,
 'train_bce_loss': 0.00655955107100887,
 'train_dice_loss': 0.006987795928599303,
 'train_time': datetime.timedelta(seconds=304, microseconds=745808),
 'val_bce_loss': 0.004798379840043486,
 'val_dice_loss': 0.007163987588630152,
 'valid_time': datetime.timedelta(seconds=49, microseconds=747396)}


Best Threshold=0.933030, Best F1 Score=0.477


...

2023-12-14 23:16:47:rastervision: INFO - epoch: 39


Training:   0%|          | 0/25 [00:00<?, ?it/s]

Validating:   0%|          | 0/3 [00:00<?, ?it/s]

2023-12-14 23:22:31:rastervision: INFO - metrics:
{'avg_f1': 0.9246941804885864,
 'avg_precision': 0.9726691246032715,
 'avg_recall': 0.8812293410301208,
 'epoch': 39,
 'other_f1': 0.9347124695777893,
 'other_precision': 0.9982942342758179,
 'other_recall': 0.878744900226593,
 'sandmine_average_precision': 0.62445156772763,
 'sandmine_best_f1_score': 0.5844970487799361,
 'sandmine_best_threshold': 0.933953,
 'sandmine_f1': 0.3431015908718109,
 'sandmine_precision': 0.20910263061523438,
 'sandmine_recall': 0.9552586674690247,
 'train_bce_loss': 0.003355823650261281,
 'train_dice_loss': 0.006453631455416506,
 'train_time': datetime.timedelta(seconds=293, microseconds=551307),
 'val_bce_loss': 0.0026469423026634904,
 'val_dice_loss': 0.006895326432727632,
 'valid_time': datetime.timedelta(seconds=50, microseconds=86748)}


Best Threshold=0.933953, Best F1 Score=0.584


## Evaluate 

Initialize evaluation_datasets and predictor.
evaluation_datasets and validation_datasets are based on identical scenes, but have different sliding window configurations.

In [11]:
config = satmae_large_inf_config
config.tile_size = 160 #force to the same size as fine-tuning

In [12]:
from ml.learner import BinarySegmentationPredictor
from utils.rastervision_pipeline import scene_to_inference_ds

evaluation_datasets =  [
    scene_to_inference_ds(
        config, scene, full_image=True
    ) for scene in validation_scenes
]

predictor = BinarySegmentationPredictor(
    config,
    model
    )

## Alternatively: specify path to trained weights
# path_to_weights = '../out/OUTPUT_DIR/last-model.pth'
# # path_to_weights = '/data/sand_mining/checkpoints/finetuned/SatMAE-L_w15.47_b128.pth'
# predictor = BinarySegmentationPredictor(
#     config,
#     model,
#     path_to_weights,
# )

In [13]:
from utils.visualizing import raster_source_to_rgb
from tqdm.notebook import tqdm
from ml.eval_utils import evaluate_predictions

prediction_results_list = []
crop_sz = int(config.tile_size // 5) #20% of the tiles at the edges are discarded

for ds in tqdm(evaluation_datasets):
    predictions = predictor.predict_mine_probability_for_site(ds)

    rgb_img = raster_source_to_rgb(ds.scene.raster_source)
    prediction_results_list.append({
        "predictions": predictions,
        "ground_truth": ds.scene.label_source.get_label_arr(),
        "rgb_img": rgb_img,
        "name": ds.scene.id,
        "crop_sz": crop_sz,
    })

evaluation_results_dict = evaluate_predictions(prediction_results_list)

  0%|          | 0/8 [00:00<?, ?it/s]

Best Threshold=0.877441, Best F1 Score=0.501
Best Threshold=0.602539, Best F1 Score=0.661
Best Threshold=0.969727, Best F1 Score=nan
Best Threshold=0.843262, Best F1 Score=0.401
Best Threshold=0.788574, Best F1 Score=0.630
Best Threshold=0.972168, Best F1 Score=0.620
Best Threshold=0.761719, Best F1 Score=0.672
Best Threshold=0.934570, Best F1 Score=0.618
Best Threshold=0.947754, Best F1 Score=0.565


## Pick the threshold that maximizes F1 score

In [14]:
threshold = evaluation_results_dict['eval/total/best_threshold']
threshold

0.9478

Log results to Weights & Biases

In [15]:
import wandb
from ml.eval_utils import make_wandb_segmentation_masks, make_wandb_predicted_probs_images

assert wandb.run is not None

# Add lists of W&B images to dict
evaluation_results_dict.update({
    'Segmentation masks': make_wandb_segmentation_masks(prediction_results_list, threshold),
    'Predicted probabilites': make_wandb_predicted_probs_images(prediction_results_list),
})

# Log to W&B
wandb.log(evaluation_results_dict)

In [10]:
wandb.finish()

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


sandmine_average_precision,▁▁▅▄▆▆▆▄▆▆▆▇▆▆▆▅▇▇▇▇▇▇▆█▇█▇████▆▆▇▆▇▆▇▆▇
sandmine_best_f1_score,▁▁▅▄▆▆▆▄▆▇▆▇▆▆▆▆▇▇▇▇▇█▇█▇█▇████▆▆▇▆▇▇▇▇▇
sandmine_best_threshold,▆▂▆▃▁▂▅▄▄▃▅▄▄▆▄█▄▃▅▆▅▅▂▃▅▄▅▄▅▁▃█▆▇▇▆▅▃▄▆
sandmine_f1,▁▃▃▄▇▇▄▅▄▇▅▆▅▃▆▂▆▇▄▅▇▆▇▇▄▇▆▆▅█▇▁▆▃▅▄▅█▆▅
sandmine_precision,▁▃▂▄▇▆▄▅▄▆▄▆▅▃▅▂▆▇▄▄▆▅▇▇▄▆▆▅▄█▇▁▅▃▄▃▄█▅▅
sandmine_recall,▇▁▆▄▂▃▅▅▅▄▆▅▄▆▅█▃▄▆▇▄▅▃▅▇▅▆▆▆▄▄█▇▇█▆▆▄▅▇
train_bce_loss,█▅▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▂▂▁▁▂▁▁▁▁▁
train_dice_loss,█▆▄▄▄▄▄▃▃▄▃▂▂▂▃▂▃▃▂▁▂▃▃▁▂▂▂▁▂▂▂▁▂▂▁▁▂▂▁▁
train_time_seconds,█▂▁▁▁▁▁▁▁▁▁▁▁▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
val_bce_loss,█▅▅▄▂▂▃▃▃▂▃▂▃▄▂▆▂▁▃▃▂▂▂▁▃▁▂▂▂▁▁█▂▅▃▃▃▁▂▂
val_dice_loss,█▇▇▆▄▃▅▄▅▃▄▃▄▆▄▆▂▃▅▄▃▃▃▂▄▃▃▄▄▂▂▇▂▆▄▄▄▁▃▃
